### query single source postgres 
includes:
* frontend parser example
* single subquery with condition
* negation
* multi way joining
* union
* view

In [1]:
import sys
import logging
sys.path.append('../')
from hybrid_engine import HybridEngine
from datalog_parser_frontend import DatalogParserFE
#qe = HybridEngine()
qe = HybridEngine(
                postgres= {'server': 'localhost', 'port': 5432, 'database': 'SQLBook', 'user': 'postgres', 'password': ''},
                asterix= {'server': 'localhost', 'port': 19002, 'dataverse': 'TinySocial'},
                solr= {'server': 'localhost', 'port': 8983, 'core': 'bookstore'})

In [2]:
# front-end parser used to convert datalog string into a structure that's agreed  as I/F with schema integration
datalog = '''
myview(numunits, firstname, billdate, orderid, customerid) <-
    postgres.orders(orderid, customerid, campaignId, orderDate, city, state, zipCode, paymentType, totalPrice, numOrderLines, _),
    postgres.customers(customerid, householdId, gender, firstname),
    postgres.orderlines(orderLineId, orderid, productid, shipDate, billdate, unitPrice, numunits, _),
    postgres.products(productid, name, groupcode, groupname, isinstock, fullprice, asin, nodeid),
    orderid > 1000, numunits > 1
Ans(customerid,numunits,firstname) <-
    view.myview(numunits,firstname,_,_,customerid)
'''
fe = DatalogParserFE()
result = fe.parse(datalog)
result

[{'condition': ['orderid > 1000', ' numunits > 1'],
  'result': 'myview(numunits,firstname,billdate,orderid,customerid)',
  'table': ['postgres.orders(orderid, customerid, campaignId, orderDate, city, state, zipCode, paymentType, totalPrice, numOrderLines, _)',
   'postgres.customers(customerid, householdId, gender, firstname)',
   'postgres.orderlines(orderLineId, orderid, productid, shipDate, billdate, unitPrice, numunits, _)',
   'postgres.products(productid, name, groupcode, groupname, isinstock, fullprice, asin, nodeid)']},
 {'condition': [],
  'result': 'Ans(customerid,numunits,firstname)',
  'table': ['view.myview(numunits,firstname,_,_,customerid)']}]

In [3]:
# single subquery with condition
datalog = '''
Ans(productid, fullprice) <- postgres.products(productid, _, _, _, _, fullprice, _, _)
                             fullprice > 100
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.DEBUG)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': ['fullprice > 100'],
  'result': 'Ans(productid,fullprice)',
  'table': ['postgres.products(productid, _, _, _, _, fullprice, _, _)']}]

<qe.DatalogParser> parser structure:
{'_result': 'Ans(productid,fullprice)',
 'aggregation': {},
 'column_to_table': {'_': {'postgres': 'products'},
                     'fullprice': {'postgres': 'products'},
                     'productid': {'postgres': 'products'}},
 'conditions': ['fullprice > 100'],
 'groupby': None,
 'join_columns': {},
 'join_path': {'MULTI_SOURCE': {}},
 'limit': None,
 'orderby': None,
 'query_columns': {'MULTI_SOURCE': [{'alias': None,
                                     'column': 'productid',
                                     'func': None,
                                     'table': 'products'},
                                    {'alias': None,
                                     'column': 'fullprice',
                                     'func': None,
               

   productid fullprice
0      10012   $195.00
1      10013   $195.00
2      10014   $175.00
3      10015   $195.00
4      10016   $195.00


(3205, 2)

In [12]:
# negation
# variable name doesn't need to be same with real column name
datalog = '''
Ans(cid, oid, value) <-
  postgres.orders(oid, cid, _, _, _, _, _, _, value, _, _),
  postgres.customers(cid, _, gender, firstname),
  not postgres.customers(cid, _, _, 'DANIEL'),
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.INFO)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': [],
  'result': 'Ans(cid,oid,value)',
  'table': ['postgres.orders(oid, cid, _, _, _, _, _, _, value, _, _)',
            'postgres.customers(cid, _, gender, firstname)']}]

<qe.PostgresEngine> query sql cmd:
SELECT customers.customerid as cid, orders.orderid as oid, orders.totalprice as value
FROM orders, customers
WHERE orders.customerid=customers.customerid

<qe.Combiner> combining result for single subquery


      cid      oid   value
0  130980  1002857  $10.00
1   21927  1003046  $17.50
2  130971  1003075  $59.80
3    7117  1003076  $22.50
4  103152  1003494  $36.00


(189559, 3)

In [5]:
# 3 way joining, condition and order
datalog = '''
Ans(customerid, productid, numunits, billdate) <- 
postgres.orders(oid, customerid, campaignId, orderDate, city, state, zipCode, paymentType, totalPrice, numOrderLines, numUnits),
postgres.customers(customerid, householdId, gender, firstname),
postgres.orderlines(orderLineId, oid, productid, shipDate, billdate, unitPrice, numunits, totalPrice),
oid > 1000, numunits > 1,
orderby numunits DESC
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.INFO)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': ['oid > 1000', ' numunits > 1'],
  'orderby': 'numunits DESC',
  'result': 'Ans(customerid,productid,numunits,billdate)',
  'table': ['postgres.orders(oid, customerid, campaignId, orderDate, city, '
            'state, zipCode, paymentType, totalPrice, numOrderLines, numUnits)',
            'postgres.customers(customerid, householdId, gender, firstname)',
            'postgres.orderlines(orderLineId, oid, productid, shipDate, '
            'billdate, unitPrice, numunits, totalPrice)']}]

<qe.PostgresEngine> query sql cmd:
SELECT customers.customerid, orderlines.productid, orderlines.numunits, orderlines.billdate
FROM orders, customers, orderlines
WHERE orders.orderid=orderlines.orderid AND orders.customerid=customers.customerid AND orders.totalprice=orderlines.totalprice AND orderlines.numunits > '1'
ORDER BY numunits DESC 

<qe.Combiner> combining result for single subquery


   customerid  productid  numunits    billdate
0      101779      11176      3000  2011-01-18
1      126295      13998      2735  2016-08-10
2       84909      10955      1500  2011-08-22
3      126295      13201      1500  2016-08-10
4      126329      13471      1419  2016-07-27


(14207, 4)

In [6]:
# predefined view: seasonal_percentages
datalog = '''
ans(pid, spring, summer, fall, winter) <- postgres.seasonal_percentages(pid, spring, summer, fall, winter)
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.WARNING)
print(result[:5])
result.shape

     pid  spring  summer   fall  winter
0  10501   50.00    50.0   0.00    0.00
1  10295   52.00    24.0   8.00   16.00
2  11523   50.00     0.0  50.00    0.00
3  11534   60.00     0.0   0.00   40.00
4  12831   11.38    12.2  35.77   40.65


(3990, 5)

In [7]:
# builtin view cooccurrence_matrix
datalog = '''
Ans(product_a, product_b, pair_count) <- postgres.cooccurrence_matrix(product_a, product_b, pair_count)
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.INFO)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': [],
  'result': 'Ans(product_a,product_b,pair_count)',
  'table': ['postgres.cooccurrence_matrix(product_a, product_b, pair_count)']}]

<qe.PostgresEngine> query sql cmd:
WITH customer_product as (
SELECT c.customerid, pr.productid, 1 as likes, c.gender as gender, o.orderdate, o.state as orderstate, o.zipcode as zipcode
FROM customers c, orders o, orderlines ol, products pr
WHERE o.customerid = c.customerid and ol.orderid = o.orderid and pr.productid = ol.productid
),
cooccurrence_matrix as (
SELECT cp1.productid as product_a, cp2.productid as product_b, count(cp1.customerid) as paircount
FROM customer_product cp1, customer_product cp2
WHERE cp1.customerid = cp2.customerid AND cp1.productid < cp2.productid
GROUP BY cp1.productid, cp2.productid
ORDER BY COUNT(cp1.customerid)  DESC
)
SELECT cooccurrence_matrix.product_a, cooccurrence_matrix.product_b, cooccurrence_matrix.paircount as pair_count
FROM cooccurrence_matrix

<qe.Combiner> combin

   product_a  product_b  pair_count
0      12820      13190        2582
1      11048      11196        1852
2      12819      12820        1846
3      10956      12139        1484
4      12139      12820        1323


(56462, 3)

In [8]:
# view by datalog
datalog = '''
cust_prod_view(cid, pid, nunits) <- 
  postgres.orders(oid, cid, _, _, _, _, _, _, _, _, _),
  postgres.orderlines(olid, oid, pid, _, date, _, nunits, price),
  postgres.products(pid, _, _, _, _, _, asin, nodeid),
  postgres.customers(cid, _, _, _)
ans(cid, pid, nunits) <- view.cust_prod_view(cid, pid, nunits), nunits > 1, orderby nunits DESC
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.INFO)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': [],
  'result': 'cust_prod_view(cid,pid,nunits)',
  'table': ['postgres.orders(oid, cid, _, _, _, _, _, _, _, _, _)',
            'postgres.orderlines(olid, oid, pid, _, date, _, nunits, price)',
            'postgres.products(pid, _, _, _, _, _, asin, nodeid)',
            'postgres.customers(cid, _, _, _)']},
 {'condition': ['nunits > 1'],
  'orderby': 'nunits DESC',
  'result': 'ans(cid,pid,nunits)',
  'table': ['view.cust_prod_view(cid, pid, nunits)']}]

<qe.PostgresEngine> query sql cmd:
WITH cust_prod_view as (SELECT customers.customerid as cid, products.productid as pid, orderlines.numunits as nunits
FROM orders, orderlines, products, customers
WHERE products.productid=orderlines.productid AND orders.orderid=orderlines.orderid AND orders.customerid=customers.customerid)
SELECT cust_prod_view.cid, cust_prod_view.pid, cust_prod_view.nunits
FROM cust_prod_view
WHERE cust_prod_view.nunits > '1'
ORDER BY nunits DESC 

<qe.Combiner> comb

      cid    pid  nunits
0  101779  11176    3000
1  126295  13998    2735
2  126295  13201    1500
3   84909  10955    1500
4  126329  13471    1419


(19558, 3)

In [9]:
# view chain
# aggregation with group by 2 columns (state, gender)
datalog = '''
view1(olid, oid, nunits) <- 
  postgres.products(pid, _, _, _, _, _, asin, nodeid),
  postgres.orderlines(olid, oid, pid, _, date, _, nunits, price),
  date > '2015-01-01'
view2(state, cid, nunits) <- 
  view.view1(olid, oid, nunits),
  postgres.orders(oid, cid, _, _, _, state, _, _, _, _, _),
view3(state, gender, nunits) <-   
  view.view2(state, cid, nunits), 
  postgres.customers(cid, _, gender, _),
Ans(state, gender, total_units) <-
  setof({nunits}, view3(state, gender, nunits), S), sum(nunits, total_units)
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.INFO)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': ["date > '2015-01-01'"],
  'result': 'view1(olid,oid,nunits)',
  'table': ['postgres.products(pid, _, _, _, _, _, asin, nodeid)',
            'postgres.orderlines(olid, oid, pid, _, date, _, nunits, price)']},
 {'condition': [],
  'result': 'view2(state,cid,nunits)',
  'table': ['view.view1(olid, oid, nunits)',
            'postgres.orders(oid, cid, _, _, _, state, _, _, _, _, _)']},
 {'condition': [],
  'groupby': {'aggregation': ['sum(nunits,total_units)'],
              'key': 'state,gender'},
  'result': 'Ans(state,gender,total_units)',
  'table': ['view.view2(state, cid, nunits)',
            'postgres.customers(cid, _, gender, _)']}]

<qe.PostgresEngine> query sql cmd:
WITH view1 as (SELECT orderlines.orderlineid as olid, orderlines.orderid as oid, orderlines.numunits as nunits
FROM products, orderlines
WHERE products.productid=orderlines.productid AND orderlines.billdate > '2015-01-01'),
view2 as (SELECT orders.state, orders.custom

  state gender  total_units
0    ND      M            9
1    AP                  10
2    WA      M          756
3    RI                  34
4    CT      M         1748


(171, 3)

In [10]:
# groupby without aggregation: distinct
datalog = '''
cust_prod_view(cid, pid, oid, price) <-
  postgres.orders(oid, cid, _, _, _, _, _, _, _, _, _),
  postgres.orderlines(olid, oid, pid, _, date, _, nunits, price),
  postgres.products(pid, _, _, _, _, _, asin, nodeid),
  postgres.customers(cid, _, _, _),
  nunits > 1
ans(pid) <-
  setof({cid}, {oid}, {price}, cust_prod_view(cid, pid, oid, price), S), count(oid, total_order)
'''
result = qe.queryDatalogRaw(datalog, loglevel=logging.INFO)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': ['nunits > 1'],
  'groupby': {'aggregation': ['count(oid,total_order)'], 'key': 'pid'},
  'result': 'ans(pid)',
  'table': ['postgres.orders(oid, cid, _, _, _, _, _, _, _, _, _)',
            'postgres.orderlines(olid, oid, pid, _, date, _, nunits, price)',
            'postgres.products(pid, _, _, _, _, _, asin, nodeid)',
            'postgres.customers(cid, _, _, _)']}]

<qe.PostgresEngine> query sql cmd:
SELECT products.productid as pid
FROM orders, orderlines, products, customers
WHERE products.productid=orderlines.productid AND orders.orderid=orderlines.orderid AND orders.customerid=customers.customerid AND orderlines.numunits > '1'
GROUP BY products.productid 

<qe.Combiner> combining result for single subquery


     pid
0  12831
1  13147
2  13331
3  11082
4  11159


(804, 1)

In [11]:
# groupby and aggregation on 2 columns
datalog = '''
cust_prod_view(cid, pid, oid, price) <-
  postgres.orders(oid, cid, _, _, _, _, _, _, _, _, _),
  postgres.orderlines(olid, oid, pid, _, date, _, nunits, price),
  postgres.products(pid, _, _, _, _, _, asin, nodeid),
  postgres.customers(cid, _, _, _),
  nunits > 1
ans(pid, total_order, total_value) <-
  setof({cid}, {oid}, {price}, cust_prod_view(cid, pid, oid, price), S), count(oid, total_order), sum(price, total_value)
'''

#DatalogParserFE().parse(datalog)
result = qe.queryDatalogRaw(datalog, loglevel=logging.INFO)
print(result[:5])
result.shape

<qe.HybridEngine> query datalog:
[{'condition': ['nunits > 1'],
  'groupby': {'aggregation': ['count(oid,total_order)',
                              'sum(price,total_value)'],
              'key': 'pid'},
  'result': 'ans(pid,total_order,total_value)',
  'table': ['postgres.orders(oid, cid, _, _, _, _, _, _, _, _, _)',
            'postgres.orderlines(olid, oid, pid, _, date, _, nunits, price)',
            'postgres.products(pid, _, _, _, _, _, asin, nodeid)',
            'postgres.customers(cid, _, _, _)']}]

<qe.PostgresEngine> query sql cmd:
SELECT products.productid as pid, count(orderlines.orderid) as total_order, sum(orderlines.totalprice) as total_value
FROM orders, orderlines, products, customers
WHERE products.productid=orderlines.productid AND orders.orderid=orderlines.orderid AND orders.customerid=customers.customerid AND orderlines.numunits > '1'
GROUP BY products.productid 

<qe.Combiner> combining result for single subquery


     pid  total_order total_value
0  12831            5     $256.85
1  13147            2      $29.98
2  13331            1       $0.00
3  11082            4     $125.43
4  11159           19     $361.54


(804, 3)